## Standard Style Transfer

In [ ]:
from styleTransfer import StyleTransferLinear
import utils

content_fn = "contents/image-1.png"
style_fn = "styles/style-0.jpg"

output_fn = "outputs/output-1.png"

content = utils.loadImage(content_fn)
style = utils.loadImage(style_fn)

stylizer = StyleTransferLinear(device="cuda")

output = stylizer(content, style)

utils.plotImage(output)
utils.saveImage(output, output_fn)


## Masked Style Transfer

In [ ]:
from styleTransfer import StyleTransferLinear
import utils

content_fn = "contents/FlyingBird.jpg"
style_fn = "styles/style-10.jpg"
mask_fn = "contents/FlyingBird_mask.jpg"

output_fn = "outputs/output-1.png"
output_fn2 = "outputs/output-2.png"

content = utils.loadImage(content_fn)
style = utils.loadImage(style_fn)
mask = utils.loadMask(mask_fn)

stylizer = StyleTransferLinear(device="cuda")

# style-then-mask
output = stylizer(content, style, mask)

# mask-then-style
output2 = stylizer(content, style, mask, premask=True)


utils.plotImage(output)
utils.plotImage(output2)

utils.saveImage(output, output_fn)
utils.saveImage(output2, output_fn2)


## Partial Convolution Style Transfer

In [ ]:
from styleTransfer import StyleTransferPartialConv
import utils

content_fn = "contents/FlyingBird.jpg"
style_fn = "styles/style-10.jpg"
mask_fn = "contents/FlyingBird_mask.jpg"

output_fn = "outputs/output-1.png"

content = utils.loadImage(content_fn)
style = utils.loadImage(style_fn)
mask = utils.loadMask(mask_fn)

stylizer = StyleTransferPartialConv(device="cuda")

# partial convolution
output = stylizer(content, style, mask)

utils.plotImage(output)

utils.saveImage(output, output_fn)


## SAM Mask Selection and Style Transfer

In [ ]:
import SAM
from styleTransfer import StyleTransferLinear, StyleTransferPartialConv
import utils

content_fn = "contents/swan.jpg"
style_fn = "styles/style-10.jpg"

output_fn = "outputs/output-1.png"
output_fn2 = "outputs/output-2.png"

content = utils.loadImage(content_fn)
style = utils.loadImage(style_fn)


stylizer_ref = StyleTransferLinear(device="cuda")
stylizer = StyleTransferPartialConv(device="cuda")
sam = SAM.SAMGenerator(device="cuda")


masks_data = sam(content)
indices = SAM.interactiveSelect(content, masks_data)
masks = SAM.loadIndices(masks_data, indices)
mask = SAM.mergeMasks(masks)

result1 = stylizer_ref(content, style, mask)
result2 = stylizer(content, style, mask)

utils.plotImage(result1)
utils.saveImage(result1, output_fn)
utils.plotImage(result2)
utils.saveImage(result2, output_fn)

In [ ]:
#style-then-mask and mask-then-style visualizations

result_stm_before = stylizer_ref(content, style, mask, postmask=False)
result_stm_after = stylizer_ref(content, style, mask, postmask=True)
result_mts_before = stylizer_ref(content, style, mask, premask=True, postmask=False)
result_mts_after = stylizer_ref(content, style, mask, premask=True, postmask=True)
utils.plotImage(result_stm_before)
utils.plotImage(result_stm_after)
utils.plotImage(result_mts_before)
utils.plotImage(result_mts_after)
utils.saveImage(result_stm_before, output_fn2.replace(".png", "-stm-before.png"))
utils.saveImage(result_stm_after, output_fn2.replace(".png", "-stm-after.png"))
utils.saveImage(result_mts_before, output_fn2.replace(".png", "-mts-before.png"))
utils.saveImage(result_mts_after, output_fn2.replace(".png", "-mts-after.png"))

import numpy as np
masked_content = utils.tensorToImage(content) * np.expand_dims(utils.tensorToMask(mask),axis=-1)
utils.plotImage(masked_content)
utils.saveImage(masked_content, output_fn2.replace(".png", "-masked-content.png"))

## Running Metrics

In [ ]:
# Between image and masked region

import numpy as np
import metrics
import matplotlib.pyplot as plt

import importlib
importlib.reload(metrics)

content_np = utils.tensorToImage(content)
mask_np = utils.tensorToMask(mask)

color_range = metrics.computeColorRange(content_np)
color_range_masked = metrics.computeColorRange(content_np, mask_np)
print("Color range (unmasked):", color_range)
print("Color range (masked):", color_range_masked)

color_average = metrics.computeColorAverage(content_np)
color_average_masked = metrics.computeColorAverage(content_np, mask_np)
print("Color average (unmasked):", color_average)
print("Color average (masked):", color_average_masked)

color_std = metrics.computeColorSTD(content_np)
color_std_masked = metrics.computeColorSTD(content_np, mask_np)
print("Color standard deviation (unmasked):", color_std)
print("Color standard deviation (masked):", color_std_masked)

color_skew = metrics.computeColorSkewness(content_np)
color_skew_masked = metrics.computeColorSkewness(content_np, mask_np)
print("Color skewness (unmasked):", color_skew)
print("Color skewness (masked):", color_skew_masked)

color_kurt = metrics.computeColorKurtosis(content_np)
color_kurt_masked = metrics.computeColorKurtosis(content_np, mask_np)
print("Color kurtosis (unmasked):", color_kurt)
print("Color kurtosis (masked):", color_kurt_masked)

color_emd = metrics.computeEarthMoversDistance(content_np, content_np, None, mask_np)
print("Color EMD" , color_emd)

color_sliced = metrics.computeSlicedWassersteinDistance(content_np, content_np, None, mask_np)
print("Color Sliced EMD" , color_sliced)

color_hist = metrics.computeColorHistogram(content_np)
color_hist_masked = metrics.computeColorHistogram(content_np, mask_np)

color_hist = color_hist.astype(np.float32)/np.amax(color_hist, axis=1, keepdims=True)  # Normalize histogram
color_hist_masked = color_hist_masked.astype(np.float32)/np.amax(color_hist_masked, axis=1, keepdims=True)  # Normalize histogram

# Plot the RGB color histograms
fig, ax = plt.subplots(2,3,figsize=(15, 5))
ax[0,0].bar(np.arange(256), color_hist[0], color='r', alpha=0.5, label='Red')
ax[0,1].bar(np.arange(256), color_hist[1], color='g', alpha=0.5, label='Green')
ax[0,2].bar(np.arange(256), color_hist[2], color='b', alpha=0.5, label='Blue')

ax[1,0].bar(np.arange(256), color_hist_masked[0], color='r', alpha=0.5, label='Red')
ax[1,1].bar(np.arange(256), color_hist_masked[1], color='g', alpha=0.5, label='Green')
ax[1,2].bar(np.arange(256), color_hist_masked[2], color='b', alpha=0.5, label='Blue')
fig.show()



In [ ]:
style_np = utils.tensorToImage(style)
result1_np = utils.tensorToImage(result1)
result2_np = utils.tensorToImage(result2)

# Between style, style-then-mask, and partial conv
color_range_style = metrics.computeColorRange(style_np)
color_range_result1 = metrics.computeColorRange(result1_np, mask_np)
color_range_result2 = metrics.computeColorRange(result2_np, mask_np)
print("Color range (style):", color_range_style)
print("Color range (style-then-mask):", color_range_result1)
print("Color range (partial conv):", color_range_result2)

color_average_style = metrics.computeColorAverage(style_np)
color_average_result1 = metrics.computeColorAverage(result1_np, mask_np)
color_average_result2 = metrics.computeColorAverage(result2_np, mask_np)
print("Color average (style):", color_average_style)
print("Color average (style-then-mask):", color_average_result1)
print("Color average (partial conv):", color_average_result2)

color_std_style = metrics.computeColorSTD(style_np)
color_std_result1 = metrics.computeColorSTD(result1_np, mask_np)
color_std_result2 = metrics.computeColorSTD(result2_np, mask_np)
print("Color standard deviation (style):", color_std_style)
print("Color standard deviation (style-then-mask):", color_std_result1)
print("Color standard deviation (partial conv):", color_std_result2)

color_emd_result1 = metrics.computeEarthMoversDistance(style_np, result1_np, None, mask_np)
color_emd_result2 = metrics.computeEarthMoversDistance(style_np, result2_np, None, mask_np)
print("Color EMD (style-then-mask):", color_emd_result1)
print("Color EMD (partial conv):", color_emd_result2)

color_sliced_result1 = metrics.computeSlicedWassersteinDistance(style_np, result1_np, None, mask_np)
color_sliced_result2 = metrics.computeSlicedWassersteinDistance(style_np, result2_np, None, mask_np)
print("Color Sliced EMD (style-then-mask):", color_sliced_result1)
print("Color Sliced EMD (partial conv):", color_sliced_result2)

color_hist_style = metrics.computeColorHistogram(style_np)
color_hist_result1 = metrics.computeColorHistogram(result1_np, mask_np)
color_hist_result2 = metrics.computeColorHistogram(result2_np, mask_np)

color_hist_style = color_hist_style.astype(np.float32)/np.amax(color_hist_style, axis=1, keepdims=True)  # Normalize histogram
color_hist_result1 = color_hist_result1.astype(np.float32)/np.amax(color_hist_result1, axis=1, keepdims=True)  # Normalize histogram
color_hist_result2 = color_hist_result2.astype(np.float32)/np.amax(color_hist_result2, axis=1, keepdims=True)  # Normalize histogram

# Mask out white and black pixels in the histograms
color_hist_style[0][0] = 0
color_hist_style[1][0] = 0
color_hist_style[2][0] = 0
color_hist_result1[0][0] = 0
color_hist_result1[1][0] = 0
color_hist_result1[2][0] = 0
color_hist_result2[0][0] = 0
color_hist_result2[1][0] = 0
color_hist_result2[2][0] = 0
color_hist_style[0][-1] = 0
color_hist_style[1][-1] = 0
color_hist_style[2][-1] = 0
color_hist_result1[0][-1] = 0
color_hist_result1[1][-1] = 0
color_hist_result1[2][-1] = 0
color_hist_result2[0][-1] = 0
color_hist_result2[1][-1] = 0
color_hist_result2[2][-1] = 0

# Plot the RGB color histograms
fig, ax = plt.subplots(3,3,figsize=(15, 5))
ax[0,0].bar(np.arange(256), color_hist_style[0], color='r', alpha=0.5, label='Red')
ax[0,1].bar(np.arange(256), color_hist_style[1], color='g', alpha=0.5, label='Green')
ax[0,2].bar(np.arange(256), color_hist_style[2], color='b', alpha=0.5, label='Blue')
ax[1,0].bar(np.arange(256), color_hist_result1[0], color='r', alpha=0.5, label='Red')
ax[1,1].bar(np.arange(256), color_hist_result1[1], color='g', alpha=0.5, label='Green')
ax[1,2].bar(np.arange(256), color_hist_result1[2], color='b', alpha=0.5, label='Blue')
ax[2,0].bar(np.arange(256), color_hist_result2[0], color='r', alpha=0.5, label='Red')
ax[2,1].bar(np.arange(256), color_hist_result2[1], color='g', alpha=0.5, label='Green')
ax[2,2].bar(np.arange(256), color_hist_result2[2], color='b', alpha=0.5, label='Blue')
#ax[0,0].set_title("Style Red")
#ax[0,1].set_title("Style Green")
#ax[0,2].set_title("Style Blue")
#ax[1,0].set_title("Style-then-mask Red")
#ax[1,1].set_title("Style-then-mask Green")
#ax[1,2].set_title("Style-then-mask Blue")
#ax[2,0].set_title("Partial Conv Red")
#ax[2,1].set_title("Partial Conv Green")
#ax[2,2].set_title("Partial Conv Blue")
fig.show()

In [ ]:
import importlib
import metrics
importlib.reload(metrics)

#print(result1.shape, result2.shape, style.shape, mask.shape)
#print(result1.dtype, result2.dtype, style.dtype, mask.dtype)

# Compute perceptual style loss
style_loss_result1 = metrics.computePerceptualStyleLoss(result1, style, mask)
style_loss_result2 = metrics.computePerceptualStyleLoss(result2, style, mask)
print("Perceptual style loss (style-then-mask):", style_loss_result1)
print("Perceptual style loss (partial conv):", style_loss_result2)

## Blending Techniques and Border Metrics

In [ ]:
## TODO: Standardize Implementation

## Mulitmask Style Transfer

In [ ]:
## TODO: STandardize Implementation